# Layouts

Layout in the context of this package describe the required content of a HDF file. A layout essentially is a HDF itself and is associates with a wrapper class, e.g. `H5File`. The idea behind defining layout is, that the data generation process is supervised. This means, when using `H5File` for filling an HDF5 file a `check()` method will tell, if the minimal required (meta) data has been written. A third user would then only accept the file if a check-run results in zero issues.

The generation and usage of layouts is outlined in the following:

In [1]:
import h5rdmtoolbox as h5tbx

The already implemented wrapper classes like e.g. `H5File` or `H5Flow` are already associated with pre-defined layouts that come with the package and we can called it directly from `H5File` as it is a property of the class:

In [2]:
with h5tbx.H5File() as h5:
    lay = h5.layout
print(lay)

<H5FileLayout with 1 issues>


As a `layout` is a HDF file itself, it has similar features like wrapper files, e.g. we can get a nice html representation of the content. Like this we can see what is required to have in any file written with `H5File`.

Doing so, we directly see, that the layout **requires to have two attributes**, namely `__h5rdmtoobox_version__` and a `title`:

In [3]:
lay

<H5FileLayout with 1 issues>

## Check layout
Let's create an empty HDF5 file with `h5py`, thus no data will be available. If we then open it with a `H5File` wrapper, everything defined in the layout file is missing, hence, we expect 4 issues to raise:

In [4]:
# create an HDF5 file with the package h5py:
import h5py
filename = h5tbx.generate_temporary_filename()
with h5py.File(filename, 'w'):
    pass

#open with h5tbx and perform a check:
with h5tbx.H5File(filename) as h5:
    print('run check')
    h5.check(silent=False)
    
    print('\nrun check again - different call')
    h5.layout.check(h5['/'], silent=False)
    
print('\nrepr of layout:')
print(h5.layout)

run check
Attribute title missing in group /

run check again - different call
Attribute title missing in group /

repr of layout:
<H5FileLayout with 1 issues>


## Write an individual layout

Layout file are HDF file, thus we need a filename an then initialize a `Layout` object:

In [5]:
hdf_layout_filename = h5tbx.generate_temporary_filename(suffix='.hdf')
mylayout = h5tbx.conventions.H5Layout(hdf_layout_filename)
mylayout

<H5FileLayout with 0 issues>

Say we expect HDF files to have the following data and structure:
 - a root attribtue "title", which an have any value
 - the datasets "x" and "y" which are 1d-arrays, which must have the attributes "standard_name" or "long_name"
 - a group "meta"

Fill the file with content. Note, the layout file is an HDF file:

In [6]:
with mylayout.File(mode='w') as h5:
    h5.attrs['title'] = '__any'
    dsx = h5.create_dataset('x', shape=(1,))  # shape does not matter here, but has to be passed
    dsx.attrs['__ndim__'] = 1
    dsx.attrs['standard_name.alt:long_name'] = '__any'
    dsy = h5.create_dataset('y', shape=(1,))  # shape does not matter here, but has to be passed
    dsx.attrs['__shape__'] = (20, )
    dsy.attrs['standard_name.alt:long_name'] = '__any'
    h5.create_group('meta')
    h5.create_group('meta/other')
mylayout

<H5FileLayout with 0 issues>

Now let's create an example HDF5 file:

In [7]:
test_filename = h5tbx.generate_temporary_filename()
with h5py.File(test_filename, mode='w') as h5:
    h5.attrs['title'] = 'my test file'
    dsx = h5.create_dataset('x', data=[1,2,3])
    dsx.attrs['long_name'] = 'x coordinate'
    dsx.attrs['standard_name'] = 'x_coordinate'
    dsy = h5.create_dataset('y', data=[[1,2],[3,4]])

We use `check()` and pass the opened hdf5 group to check for any issues:

In [8]:
with h5py.File(test_filename, mode='r') as h5:
    mylayout.check(h5, silent=False)

Group name "meta" missing in /.
Wrong shape of dataset /x: (20,) != (3,)
Neither of the attribute standard_name, long_name exist in /y


We get 3 issues as stated in the printouts above. If we suppress the print statements by `silent=True`, then we can get a similar output by calling `.report()`:

In [9]:
mylayout.report()

H5FileLayout issue report (3 issues)
-------------------
/meta: -> missing
/x: -> wrong shape: (3,) != (20,)
/y.standard_name or long_name: -> missing


## Relative checking
By callng `.check()` from the layout object, we can change the entry point by passing other HDF groups than the root group, e.g. we could start from the group "meta":

In [10]:
with h5py.File(test_filename, mode='r+') as h5:
    h5.create_group('meta')
    mylayout.check(h5['meta'], silent=True)
print(mylayout)

<H5FileLayout with 1 issues>


Also we can specify, whether we want to run the check on all subgropups or only stay at the entry level. Such an recursive search can be suppressed by passing `recursive=False`:

In [11]:
with h5py.File(test_filename, mode='r+') as h5:
    mylayout.check(h5['/'], silent=True, recursive=False)
print(mylayout)

<H5FileLayout with 2 issues>
